In [14]:
import pandas as pd
import pyarrow

anime = pd.read_csv("anime.csv")
rating = pd.read_csv("rating.csv")

anime.to_parquet('anime.parquet')
rating.to_parquet('rating.parquet')



In [15]:
rating.shape

(7813737, 3)

In [16]:
anime.isnull().sum()

Unnamed: 0      0
anime_id        0
name            0
genre          62
type           25
episodes        0
rating        230
members         0
dtype: int64

In [17]:
rating.isnull().sum()

user_id     0
anime_id    0
rating      0
dtype: int64

In [18]:
anime.duplicated().sum()

0

In [19]:
rating.duplicated().sum()

1

In [20]:
merged = rating.merge(anime,on='anime_id')

In [21]:
merged

,user_id,anime_id,rating_x,Unnamed: 0,name,genre,type,episodes,rating_y,members
0,1,20,-1,841,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
1,3,20,8,841,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
2,5,20,6,841,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
3,6,20,-1,841,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
4,10,20,-1,841,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
...,...,...,...,...,...,...,...,...,...,...
7813722,65682,30450,8,8493,Dr. Slump: Hoyoyo! Arale no Himitsu Dai Koukai...,"Comedy, Sci-Fi, Shounen",Special,1,6.17,248
7813723,69497,33484,10,10256,Shiroi Zou,"Action, Historical, Kids",Movie,1,4.71,45
7813724,70463,29481,-1,9097,Kakinoki Mokkii,"Fantasy, Kids",Special,1,4.33,61
7813725,72404,34412,-1,8777,Hashiri Hajimeta bakari no Kimi ni,Music,Music,1,6.76,239


In [22]:
numr = merged.groupby('anime_id').count()['rating_x'].reset_index()
numr.rename(columns={'rating_x':'num_ratings'},inplace=True)
numr

,anime_id,num_ratings
0,1,15509
1,5,6927
2,6,11077
3,7,2629
4,8,413
...,...,...
11192,34367,5
11193,34412,1
11194,34475,4
11195,34476,1


In [23]:
numr.head(5000)

,anime_id,num_ratings
0,1,15509
1,5,6927
2,6,11077
3,7,2629
4,8,413
...,...,...
4995,7430,73
4996,7435,13
4997,7436,14
4998,7445,9


In [24]:
x = merged.groupby('user_id').count()['rating_x'] > 150

In [25]:
cultured = x[x].index

filtered = merged[merged['user_id'].isin(cultured)]

y = filtered.groupby('anime_id').count()['rating_x']>=150
eanime = y[y].index

final = filtered[filtered['anime_id'].isin(eanime)]

lfinal = final.pivot_table(index='anime_id',columns='user_id',values='rating_x')

lfinal.fillna(0,inplace=True)

In [26]:
lfinal.to_parquet('lfinal.parquet')



In [27]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_scores = cosine_similarity(lfinal)

similarity_scores.shape

(3691, 3691)

In [45]:
import numpy as np
similarity_scores
similarity_scores = pd.DataFrame(similarity_scores)
similarity_scores.to_parquet('user_similarity_scores.parquet')
similarity_scores = similarity_scores.values


In [29]:

lfinal.index

Index([    1,     5,     6,     7,     8,    15,    16,    17,    18,    19,
       ...
       33201, 33222, 33338, 33421, 33524, 33558, 33569, 33964, 34103, 34240],
      dtype='int64', name='anime_id', length=3691)

In [30]:
def get_recommendation(anime_id):
    index = np.where(lfinal.index==anime_id)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:50]
    data = []
    for i in similar_items:
        temp_df = anime[anime['anime_id'] == lfinal.index[i[0]]]
        anime_id = temp_df.drop_duplicates('anime_id')['anime_id'].values[0]
        data.append(anime_id)
    return data[:27]

In [32]:
get_recommendation(20)

[1535,
 1575,
 11757,
 269,
 121,
 5114,
 16498,
 9919,
 2904,
 8074,
 226,
 3588,
 6547,
 442,
 813,
 4224,
 2472,
 6702,
 10620,
 936,
 6880,
 4437,
 223,
 2167,
 356,
 6746,
 2144]

In [33]:
get_recommendation(1735)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [37]:
anime['genre'] = anime['genre'].apply(lambda x: str(x).split(", "))
genre_encoded_df = anime['genre'].str.join('|').str.get_dummies()

anime_encoded = pd.concat([anime, genre_encoded_df], axis=1)
anime_encoded

,Unnamed: 0,anime_id,name,genre,type,episodes,rating,members,Action,Adventure,...,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri,nan
0,0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",Movie,1,9.37,200630,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic, Mil...",TV,64,9.26,793665,1,1,...,0,0,0,0,0,0,0,0,0,0
2,2,28977,Gintama°,"[Action, Comedy, Historical, Parody, Samurai, ...",TV,51,9.25,114262,1,0,...,0,0,0,0,0,0,0,0,0,0
3,3,9253,Steins;Gate,"[Sci-Fi, Thriller]",TV,24,9.17,673572,0,0,...,0,0,0,0,0,1,0,0,0,0
4,4,9969,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samurai, ...",TV,51,9.16,151266,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,[Hentai],OVA,1,4.15,211,0,0,...,0,0,0,0,0,0,0,0,0,0
12290,12290,5543,Under World,[Hentai],OVA,1,4.28,183,0,0,...,0,0,0,0,0,0,0,0,0,0
12291,12291,5621,Violence Gekiga David no Hoshi,[Hentai],OVA,4,4.88,219,0,0,...,0,0,0,0,0,0,0,0,0,0
12292,12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,[Hentai],OVA,1,4.98,175,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
from sklearn.metrics.pairwise import cosine_similarity


genre_similarity_scores = cosine_similarity(genre_encoded_df)
genre_similarity_scores_df = pd.DataFrame(genre_similarity_scores, index=anime['anime_id'], columns=anime['anime_id'])
genre_similarity_scores_df.to_parquet('genre_similarity_scores_df.parquet')
genre_similarity_scores_df

anime_id,32281,5114,28977,9253,9969,32935,11061,820,15335,15417,...,26031,34399,10368,9352,5541,9316,5543,5621,6133,26081
anime_id,,,,,,,,,,,,,,,,,,,,,
32281,1.000000,0.188982,0.000000,0.000000,0.000000,0.447214,0.000000,0.250000,0.000000,0.000000,...,0.0,0.288675,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5114,0.188982,1.000000,0.285714,0.000000,0.285714,0.338062,0.566947,0.377964,0.285714,0.285714,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28977,0.000000,0.285714,1.000000,0.267261,1.000000,0.338062,0.377964,0.188982,1.000000,1.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9253,0.000000,0.000000,0.267261,1.000000,0.267261,0.000000,0.000000,0.353553,0.267261,0.267261,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9969,0.000000,0.285714,1.000000,0.267261,1.000000,0.338062,0.377964,0.188982,1.000000,1.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.577350,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5543,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.577350,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5621,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.577350,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [41]:
def genre_based_recommendation(anime_name):

    similarity_scores = genre_similarity_scores_df[anime_name]

    recommended_animes = similarity_scores.sort_values(ascending=False)[1:]

    return recommended_animes.head(27)

In [42]:
genre_based_recommendation(1735)

anime_id
20       1.000000
28755    1.000000
6325     1.000000
10659    1.000000
8246     1.000000
32365    1.000000
10075    1.000000
19511    1.000000
1604     0.894427
23933    0.894427
1009     0.894427
30694    0.845154
987      0.845154
14527    0.845154
904      0.845154
22695    0.845154
174      0.845154
25389    0.845154
11761    0.845154
6714     0.845154
22777    0.845154
813      0.845154
6033     0.845154
11787    0.800000
24029    0.800000
269      0.800000
2335     0.800000
Name: 1735, dtype: float64

In [48]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

anime = pd.read_parquet("anime.parquet")
user_similarity_scores_df = pd.read_parquet('user_similarity_scores.parquet')
genre_similarity_scores_df = pd.read_parquet('genre_similarity_scores_df.parquet')


def genre_based_recommendation(anime_id):
    similarity_scores = genre_similarity_scores_df[anime_id]
    recommended_animes = similarity_scores.sort_values(ascending=False)[1:]
    return recommended_animes.head(10).index.tolist()

def user_based_recommendation(anime_id):
    index = np.where(lfinal.index==anime_id)[0][0]
    similar_items = sorted(list(enumerate(user_similarity_scores_df[index])),key=lambda x:x[1],reverse=True)[1:50]
    data = []
    for i in similar_items:
        temp_df = anime[anime['anime_id'] == lfinal.index[i[0]]]
        anime_id = temp_df.drop_duplicates('anime_id')['anime_id'].values[0]
        data.append(anime_id)
    return data[:27]

def get_recommendation(anime_id):
    if anime_id in lfinal.index:
        return user_based_recommendation(anime_id)
    else:
        return genre_based_recommendation(anime_id)

In [49]:
print(get_recommendation(20))

[1535, 1575, 11757, 269, 121, 5114, 16498, 9919, 2904, 8074, 226, 3588, 6547, 442, 813, 4224, 2472, 6702, 10620, 936, 6880, 4437, 223, 2167, 356, 6746, 2144]


In [54]:
import pandas as pd
import requests
from sklearn.metrics.pairwise import cosine_similarity

anime = pd.read_parquet("anime.parquet")
user_similarity_scores_df = pd.read_parquet('user_similarity_scores.parquet')
genre_similarity_scores_df = pd.read_parquet('genre_similarity_scores_df.parquet')


def genre_based_recommendation(anime_id):
    similarity_scores = genre_similarity_scores_df[anime_id]
    recommended_animes = similarity_scores.sort_values(ascending=False)[1:]
    return recommended_animes.head(10).index.tolist()

def user_based_recommendation(anime_id):
    index = np.where(lfinal.index==anime_id)[0][0]
    similar_items = sorted(list(enumerate(user_similarity_scores_df[index])),key=lambda x:x[1],reverse=True)[1:50]
    data = []
    for i in similar_items:
        temp_df = anime[anime['anime_id'] == lfinal.index[i[0]]]
        anime_id = temp_df.drop_duplicates('anime_id')['anime_id'].values[0]
        data.append(anime_id)
    return data[:27]

def get_anime_info(anime_id, client_id='17d7a82e16e1c7813f3d03cceb1cf0b9'):
    base_url = f"https://api.myanimelist.net/v2/anime/{anime_id}"
    params = {
        "fields": "synopsis,main_picture,mean,genre,media_type",
    }
    headers = {
        "X-MAL-CLIENT-ID": client_id
    }
    try:   
        response = requests.get(base_url, params=params, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if data:
                return data
            else:
                print(f"No results found for anime ID '{anime_id}'.")
        else:
            print(f"Failed to fetch data from MyAnimeList API. Status code: {response.status_code}")
            print(response.text) 
    except requests.RequestException as e:
        print(f"An error occurred: {e}")

def get_recommendation(anime_id):
    if anime_id in user_similarity_scores_df.columns:
        recommend_ids = user_based_recommendation(anime_id)
        print('user')
    else:
        recommend_ids = genre_based_recommendation(anime_id)
        print('genre')
    recommend_data = []
    for id in recommend_ids:
        recommend_data.append(get_anime_info(id))
    return recommend_data


In [55]:
print(get_recommendation(20))

user
[{'id': 1535, 'title': 'Death Note', 'main_picture': {'medium': 'https://cdn.myanimelist.net/images/anime/9/9453.jpg', 'large': 'https://cdn.myanimelist.net/images/anime/9/9453l.jpg'}, 'synopsis': "Brutal murders, petty thefts, and senseless violence pollute the human world. In contrast, the realm of death gods is a humdrum, unchanging gambling den. The ingenious 17-year-old Japanese student Light Yagami and sadistic god of death Ryuk share one belief: their worlds are rotten.\n\nFor his own amusement, Ryuk drops his Death Note into the human world. Light stumbles upon it, deeming the first of its rules ridiculous: the human whose name is written in this note shall die. However, the temptation is too great, and Light experiments by writing a felon's name, which disturbingly enacts his first murder.\n\nAware of the terrifying godlike power that has fallen into his hands, Light—under the alias Kira—follows his wicked sense of justice with the ultimate goal of cleansing the world of 